importing necessary libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import nni
from nni.nas.nn.pytorch import LayerChoice, ModelSpace, MutableDropout, MutableLinear
import nni.nas.strategy as strategy
from nni.nas.strategy.middleware import Filter, Chain
from nni.nas.profiler.pytorch.flops import NumParamsProfiler
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from nni.nas.evaluator import FunctionalEvaluator
from nni.nas.experiment import NasExperiment
import time
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

c:\Users\azhar\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\transforms\_functional_pil.py:242: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  interpolation: int = Image.BILINEAR,
c:\Users\azhar\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\transforms\_functional_pil.py:286: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  interpolation: int = Image.NEAREST,
c:\Users\azhar\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\transforms\_functional_pil.py:319: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  interpolation: int = Image.BICUBIC,


Define Model Space with Model

In [2]:
class MyModelSpace(ModelSpace):
    def __init__(self):
        super().__init__()
        self.conv1 = LayerChoice([
            nn.Conv2d(1, 32, 3, 1),
            nn.Conv2d(1,32,5,1,1),],label = "conv1")
        
        self.conv2 = LayerChoice([
            nn.Conv2d(32, 64,3,1),
            nn.Conv2d(32, 64,5,1,1),
            
        ], label='conv2')
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.dropout1 = MutableDropout(nni.choice('dropout', [0.25, 0.5, 0.75]))  
        self.dropout2 = nn.Dropout(0.5)
        feature = nni.choice('feature', [64, 128, 256])
        self.fc1 = MutableLinear(9216, feature)
        self.fc2 = MutableLinear(feature, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = torch.flatten(self.dropout1(x), 1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


model_space = MyModelSpace()


Function to get memory of Model

In [3]:
def get_model_size(model):
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    num_buffers = sum(b.numel() for b in model.buffers())
    total_size = num_params + num_buffers
    return total_size * 4 / (1024 ** 2)

Function to evaluate a model and report performance metric to NNI

In [4]:
def evaluate_model(model):
    
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    transf = transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize the pixel values
])
    train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transf, download=True)
    test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transf   , download=True)


    batch_size = 64


    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)        

    for epoch in range(3):
        
        dict={"default":0.0,"accuracy":0.0,"latency":0.0,"memory":0.0}
        
        loss_fn = torch.nn.CrossEntropyLoss()
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
        
        tim1=time.time()
        
        model.eval()
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        
        accuracy = 100. * correct / len(test_loader.dataset)
        
        time2 = time.time()-tim1
        mem = get_model_size(model)
        met = ((accuracy/100)-((time2-1)/(2-1))-((mem-2)/(10-2)))*100
        dict['default']=float(met)
        dict['accuracy']=accuracy
        dict['latency']=time2
        dict["memory"]=mem
        
        nni.report_intermediate_result(dict)

    
    nni.report_final_result(dict)

Create the evaluator of NNI

In [5]:
evaluator = FunctionalEvaluator(evaluate_model)

Defining  different stratergies to evalaute

In [6]:
search_strategy1 = strategy.Random() 
search_strategy2 = strategy.RegularizedEvolution()  
search_strategy3 = strategy.TPE()  

[2024-05-05 18:06:25] Using random seed 2111768997


Defining Experiment for different stratergies

In [7]:
experiment1 = NasExperiment(model_space, evaluator, search_strategy1)
experiment2 = NasExperiment(model_space, evaluator, search_strategy2)
experiment3 = NasExperiment(model_space, evaluator, search_strategy3)

[2024-05-05 18:06:25] Config is not provided. Will try to infer.
[2024-05-05 18:06:25] Using execution engine based on training service. Trial concurrency is set to 1.
[2024-05-05 18:06:25] Using simplified model format.
[2024-05-05 18:06:25] Using local training service.
[2024-05-05 18:06:25] WARNING: GPU found but will not be used. Please set `experiment.config.trial_gpu_number` to the number of GPUs you want to use for each trial.
[2024-05-05 18:06:25] Config is not provided. Will try to infer.
[2024-05-05 18:06:25] Using execution engine based on training service. Trial concurrency is set to 1.
[2024-05-05 18:06:25] Using simplified model format.
[2024-05-05 18:06:25] Using local training service.
[2024-05-05 18:06:25] WARNING: GPU found but will not be used. Please set `experiment.config.trial_gpu_number` to the number of GPUs you want to use for each trial.
[2024-05-05 18:06:25] Config is not provided. Will try to infer.
[2024-05-05 18:06:25] Using execution engine based on train

Running experiments on differnt ports

In [8]:
experiment1.run(port=8085)

[2024-05-05 18:06:26] Creating experiment, Experiment ID: r9iv4tkb
[2024-05-05 18:06:26] Starting web server...
[2024-05-05 18:06:28] Setting up...
[2024-05-05 18:06:29] Web portal URLs: http://169.254.55.27:8085 http://169.254.97.166:8085 http://192.168.56.1:8085 http://169.254.83.81:8085 http://169.254.82.251:8085 http://169.254.92.2:8085 http://172.31.25.119:8085 http://169.254.253.36:8085 http://169.254.191.241:8085 http://127.0.0.1:8085
[2024-05-05 18:06:29] Successfully update searchSpace.
[2024-05-05 18:06:29] Checkpoint saved to C:\Users\azhar\nni-experiments\r9iv4tkb\checkpoint.
[2024-05-05 18:06:29] Experiment initialized successfully. Starting exploration strategy...
[2024-05-05 18:16:13] Waiting for models submitted to engine to finish...
[2024-05-05 18:17:16] Experiment is completed.
[2024-05-05 18:17:16] Search process is done. You can put an `time.sleep(FOREVER)` here to block the process if you want to continue viewing the experiment.


True

In [9]:
experiment2.run(port=8086)

[2024-05-05 18:17:18] Creating experiment, Experiment ID: k4gef6vw
[2024-05-05 18:17:18] Starting web server...
[2024-05-05 18:17:19] Setting up...
[2024-05-05 18:17:20] Web portal URLs: http://169.254.55.27:8086 http://169.254.97.166:8086 http://192.168.56.1:8086 http://169.254.83.81:8086 http://169.254.82.251:8086 http://169.254.92.2:8086 http://172.31.25.119:8086 http://169.254.253.36:8086 http://169.254.191.241:8086 http://127.0.0.1:8086
[2024-05-05 18:17:20] Successfully update searchSpace.
[2024-05-05 18:17:20] Checkpoint saved to C:\Users\azhar\nni-experiments\k4gef6vw\checkpoint.
[2024-05-05 18:17:20] Experiment initialized successfully. Starting exploration strategy...
[2024-05-05 18:17:20] Spawning the initial population. 100 individuals to go.
[2024-05-05 18:17:20] [Individual    1] Random: {'conv1': 0, 'conv2': 1, 'dropout': 0.5, 'feature': 128}
[2024-05-05 18:18:25] [Metric] -65.380544 Sample: {'conv1': 0, 'conv2': 1, 'dropout': 0.5, 'feature': 128}
[2024-05-05 18:18:26] [

In [ ]:
experiment3.run(port=8087)

[2024-05-05 17:26:17] Creating experiment, Experiment ID: m1xv6sn9
[2024-05-05 17:26:17] Starting web server...
[2024-05-05 17:26:18] Setting up...
[2024-05-05 17:26:19] Web portal URLs: http://169.254.55.27:8087 http://169.254.97.166:8087 http://192.168.56.1:8087 http://169.254.83.81:8087 http://169.254.82.251:8087 http://169.254.92.2:8087 http://172.31.25.119:8087 http://169.254.253.36:8087 http://169.254.191.241:8087 http://127.0.0.1:8087
[2024-05-05 17:26:19] Successfully update searchSpace.
[2024-05-05 17:26:19] Checkpoint saved to C:\Users\azhar\nni-experiments\m1xv6sn9\checkpoint.
[2024-05-05 17:26:19] Experiment initialized successfully. Starting exploration strategy...
[2024-05-05 17:38:48] Tuning algorithm generated duplicate parameter: {('conv1',): 0, ('conv2',): 1, ('dropout',): 2, ('feature',): 1}
[2024-05-05 17:38:48] Use grid search for deduplication.
[2024-05-05 17:38:48] Grid initialized, size: (2×2×3×3) = 36
